## Installing Sentence Transsformer and other models/frameworks

In [ ]:
!pip install sentence_transformers

# Kindly add all your installations and versions if any in this cell.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.7 MB/s 
     |████████████████████████████████| 5.5 MB 37.8 MB/s 
     |████████████████████████████████| 1.3 MB 45.3 MB/s 
     |████████████████████████████████| 182 kB 47.1 MB/s 
     |████████████████████████████████| 7.6 MB 48.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2bcbde524d351507596cbaf5d8f2e15393f5013812471dd93c5f5ee577b5a471
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


## Importing necessary libraries. 
In the final version all imports should be stricly enlisted here.

In [ ]:
import pandas as pd
import numpy as np
import spacy
from scipy import stats
from sklearn import linear_model
import csv
from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from gensim.models import Word2Vec
import torch 
from torch.utils.data import DataLoader
import multiprocessing
import gensim
all_stopwords = gensim.parsing.preprocessing.STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
import gensim.downloader
from sklearn import svm
from sklearn.linear_model import TweedieRegressor
from sklearn.ensemble import RandomForestRegressor
from sentence_transformers import SentenceTransformer
from sentence_transformers import evaluation
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import InputExample


## Load dataset: 7 marks
1 Download and unzip the dataset from this link http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz  **1 mark**

2 Complete the code in `read_sts_csv()`. **4.5 marks**

3 Create 3 dataframes one each for train, test and val and print their final shapes. **1.5 marks**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!tar -zxvf /content/drive/MyDrive/Stsbenchmark.tar.gz -C /content/drive/MyDrive/nlp_unzip/stsbenchmark/

stsbenchmark/
stsbenchmark/readme.txt
stsbenchmark/sts-test.csv
stsbenchmark/correlation.pl
stsbenchmark/LICENSE.txt
stsbenchmark/sts-dev.csv
stsbenchmark/sts-train.csv


In [ ]:
INPUT_PATH = '/content/drive/MyDrive/nlp_unzip/stsbenchmark/'

In [ ]:
def read_sts_csv(dataset_type="train", columns=['source', 'type', 'year', 'id', 'score', 'sent_a', 'sent_b']):
  path = INPUT_PATH + "sts-"+ dataset_type + ".csv"
  """
  Take the input path and return the dataframe
  """
  df=pd.read_csv(path,sep='\t',quoting=csv.QUOTE_NONE,names=columns)
  
  return df

df_train = read_sts_csv("train") # create the train, dev and test dataframes
df_dev = read_sts_csv("dev")
df_test = read_sts_csv("test")


## Hyperparameters: 5 Marks
Update this cell with you choosen parameters except, NUM_EPOCHS

In [ ]:
NON_CONEXTUAL_MODEL_TYPE = "Word2Vec"
CONEXTUAL_MODEL_TYPE = "all-mpnet-base-v2"
HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL = 'distilroberta-base' # USE THE HUGGAING FACE VERSION OF SENTENCE_TRANSFORMER_TYPE
INPUT_PATH = '/content/drive/MyDrive/nlp_unzip/stsbenchmark/'
BATCH_SIZE = 200
OUT_DIM_DENSE = 50
NUM_EPOCHS = 2 ## THIS IS FIXED DO NOT CHANGE

# You are free to add your own hyperparameters as well.

## CONFIGURATION 1: Non-contextual Embeddings + ML Regression: 8 marks
1 Load the non-contextual embedding model in variable `non_cont_model1`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model1()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model1`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model1`. **1.5 mark**



In [ ]:
df_train=df_train.sample(frac=1)

In [ ]:
glove_vectors = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
vectors_size=glove_vectors.vector_size

In [ ]:
##EXPERIMENTING:
sent_a_list=df_train['sent_a'].tolist()
cores = multiprocessing.cpu_count()
l=[]
for i in sent_a_list:
  i=i.lower()
  # print(i,type(i))
  i=remove_stopwords(i)
  # print(i)
  l.append(i.split(" "))

In [ ]:
model = Word2Vec(size=100, window=2, min_count=1,sample=6e-5, alpha=0.03,min_alpha=0.0007,negative=20,workers=cores-1)
model.build_vocab(l, progress_per=10000)
model.train(l, total_examples=model.corpus_count, epochs=2, report_delay=1)

(50848, 71598)

In [ ]:
model.wv.vectors.shape

(11968, 100)

In [ ]:
vector = model.wv['woman']  # get numpy vector of a word
sims = model.wv.most_similar(positive=["woman"])

In [ ]:
vector_size=(glove_vectors.vector_size)

In [ ]:
def get_feature_model1(data_frame):


  sent_a_list=data_frame['sent_a'].tolist()
  sent_b_list=data_frame['sent_b'].tolist()
  cores = multiprocessing.cpu_count()
  l1=[]
  l2=[]
  for i in range(len(sent_a_list)):
    sent_a_list[i]=sent_a_list[i].lower()
    sent_a_list[i]=remove_stopwords(sent_a_list[i])
    l1.append(sent_a_list[i].split(" "))

    sent_b_list[i]=sent_b_list[i].lower()
    sent_b_list[i]=remove_stopwords(sent_b_list[i])
    l2.append(sent_b_list[i].split(" "))
    # print(i,type(i))
    
    # print(i)

  # a=non_cont_model1.wv.vectors
  sent_a_vectors=[]
  for i in range(len(l1)):
    s_vec = np.zeros(vector_size)
    c=0
    for word in l1[i]:
      if(word in non_cont_model1.vocab):
        v = non_cont_model1.word_vec(word)
        s_vec += v
        c+=1
    if c>0:
        s_vec /= c
    sent_a_vectors.append(s_vec)

  sent_b_vectors=[]
  for i in range(len(l2)):
    s_vec = np.zeros(vector_size)
    c=0
    for word in l2[i]:
      if(word in non_cont_model1.vocab):
        v = non_cont_model1.word_vec(word)
        s_vec += v
        c+=1
    if c>0:
        s_vec /= c
    sent_b_vectors.append(s_vec)

  # b=non_cont_model1.wv.vectors
  """
  Input a data frame and return the embedding vectors for the each sentence column using non_cont_model1,
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """
  return sent_a_vectors,sent_b_vectors

non_cont_model1 = glove_vectors

feature_1_train, feature_2_train = get_feature_model1(df_train)
print(len(feature_1_train))
X_train=np.zeros(len(feature_1_train)*2*vector_size).reshape(len(feature_1_train),2*vector_size)
for i in range(len(feature_1_train)):
  X_train[i][:vector_size] = feature_1_train[i]
  X_train[i][vector_size:] = feature_2_train[i]

Y_train = df_train['score']
# Initiate a regression model and train it.



5749


In [ ]:
(X_train).shape

(5749, 600)

In [ ]:

model1 = RandomForestRegressor(max_depth=5, random_state=0)
model1.fit(X_train, Y_train)

# Print spearmanr correlation on the predicted output of the dev and test sets.
feature_1_dev, feature_2_dev = get_feature_model1(df_dev)
X_dev=np.zeros(len(feature_1_dev)*2*vector_size).reshape(len(feature_1_dev),2*vector_size)
for i in range(len(feature_1_dev)):
  X_dev[i][:vector_size] = feature_1_dev[i]
  X_dev[i][vector_size:] = feature_2_dev[i]
Y_dev = df_dev['score']

feature_1_test, feature_2_test = get_feature_model1(df_test)
X_test=np.zeros(len(feature_1_test)*2*vector_size).reshape(len(feature_1_test),2*vector_size)
for i in range(len(feature_1_test)):
  X_test[i][:vector_size] = feature_1_test[i]
  X_test[i][vector_size:] = feature_2_test[i]
Y_test = df_test['score']


dev_pred=model1.predict(X_dev)
test_pred=model1.predict(X_test)

In [ ]:
dev_pred

array([2.45220234, 2.49894326, 2.78506025, ..., 2.60383306, 2.71755452,
       2.73592565])

In [ ]:
df_dev['score']

0       5.00
1       4.75
2       5.00
3       2.40
4       2.75
        ... 
1495    2.00
1496    0.00
1497    2.00
1498    0.00
1499    0.00
Name: score, Length: 1500, dtype: float64

In [ ]:
from scipy import stats
print("For dev",stats.spearmanr(dev_pred, df_dev['score']))
print("For test",stats.spearmanr(test_pred, df_test['score']))

For dev SpearmanrResult(correlation=0.2573096978487676, pvalue=4.139305702024481e-24)
For test SpearmanrResult(correlation=0.3301018902165999, pvalue=2.0558669974719678e-36)


## CONFIGURATION 2: Contextual Embeddings + ML Regression: 7 marks
1 Load the contextual embedding model in variable `non_cont_model2`. **1 marks**

2 Get feature for the sentences using the LM model loaded before. Add the code in the `get_feature_model2()` **2 marks**

2 Using features as X and score as Y, train a ML based regression model (`model2`). You are free to choose any sklearn based regression method, and its hyperparameters. **3.5 marks**

3 Print the correlation scores on the dev and test set predictions using trained `model2`. **1.5 mark**

Useful references: https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [ ]:
!pip install -U sentence-transformers

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
v_size=0

In [ ]:
def get_feature_model2(data_frame):
  sent_a_list=data_frame['sent_a'].tolist()
  sent_b_list=data_frame['sent_b'].tolist()
  cores = multiprocessing.cpu_count()
  l1=[]
  l2=[]
  for i in range(len(sent_a_list)):
    sent_a_list[i]=sent_a_list[i].lower()
    sent_a_list[i]=remove_stopwords(sent_a_list[i])

    sent_b_list[i]=sent_b_list[i].lower()
    sent_b_list[i]=remove_stopwords(sent_b_list[i])

    
    a=model.encode(sent_a_list[i])
    b=model.encode(sent_b_list[i])
    v_size = len(a)
    l1.append(a)
    l2.append(b)

  return l1,l2

non_cont_model2 = model

feature_1_train, feature_2_train = get_feature_model2(df_train)
print(len(feature_1_train))



# feature_1_<dataset_type>, feature_2_<dataset_type> = get_feature_model2(data_frame)
# X_<dataset_type>, Y_<dataset_type> = 
# Initiate a regression model and train it.
# Print spearman correlation on the predicted output of the dev and test sets.

In [ ]:
v_size=768

In [ ]:
X_train=np.zeros(len(feature_1_train)*2*v_size).reshape(len(feature_1_train),2*v_size)
for i in range(len(feature_1_train)):
  X_train[i][:v_size] = feature_1_train[i]
  X_train[i][v_size:] = feature_2_train[i]

Y_train = df_train['score']

In [ ]:
model1 = RandomForestRegressor(max_depth=5, random_state=0)
model1.fit(X_train, Y_train)

# Print spearmanr correlation on the predicted output of the dev and test sets.
feature_1_dev, feature_2_dev = get_feature_model2(df_dev)
X_dev=np.zeros(len(feature_1_dev)*2*v_size).reshape(len(feature_1_dev),2*v_size)
for i in range(len(feature_1_dev)):
  X_dev[i][:v_size] = feature_1_dev[i]
  X_dev[i][v_size:] = feature_2_dev[i]
Y_dev = df_dev['score']

feature_1_test, feature_2_test = get_feature_model2(df_test)
X_test=np.zeros(len(feature_1_test)*2*v_size).reshape(len(feature_1_test),2*v_size)
for i in range(len(feature_1_test)):
  X_test[i][:v_size] = feature_1_test[i]
  X_test[i][v_size:] = feature_2_test[i]
Y_test = df_test['score']


dev_pred=model1.predict(X_dev)
test_pred=model1.predict(X_test)

In [ ]:
from scipy import stats
print("For dev",stats.spearmanr(dev_pred, df_dev['score']))
print("For test",stats.spearmanr(test_pred, df_test['score']))

For dev SpearmanrResult(correlation=0.47640585279411596, pvalue=7.89513463579808e-86)
For test SpearmanrResult(correlation=0.47404043351166525, pvalue=3.547460178517629e-78)


## CONFIGURATION 3: Fine-Tune a Contextual Embeddings Model: 18 marks

1 Prepare data samples to be for the DL model to consume. Add the code in the `form_data()`. **4 marks**

3 Create the data loader, one each for train/dev/test data_input sample set obtained from `form_input_example()`. **1.5 marks**

4 Initiate `model3` consisting of **atleast** the following 3 components - `base_LM`, a `pooling_layer` and a `dense_layer`. Use appropriate activation function in dense. **Atleast** one layer of `base_LM` should be set to trainable. **5 marks**

6 Initiate the `loss`. **0.5 marks**

7 Fit the `model3`. Use `NUM_EPOCHS = 2`. **MAX_NUM_EPOCHS allowed will be 3**. **2 marks** 

8 Complete the `get_model_predicts()` to obtain predicted scores for input sentence pairs. **3.5 marks** 

9 Print the correlation scores on the dev and test set predictions. **1.5 mark**

Useful References: https://huggingface.co/blog/how-to-train-sentence-transformers 

In [ ]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

## Step 1: use an existing language model
word_embedding_model = models.Transformer('distilroberta-base',do_lower_case=True,max_seq_length=50)

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=50, activation_function=nn.GELU())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])



import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 50, 'do_lower_case': True}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 50, 'bias': True, 'activation_function': 'torch.nn.modules.activation.GELU'})
)

In [ ]:

def form_data(data_frame):
  train_data=[]
  for i in range(len(data_frame)):  
    train_data.append(InputExample(texts=[data_frame['sent_a'][i], data_frame['sent_b'][i]],label= float(data_frame['score'][i])))

  # print(train_data[0])
  dataloader = DataLoader(train_data, batch_size=200, shuffle=True)
  return dataloader
  """
  Input a data frame and return the dataloder.
  """

def get_model_predicts(data_type, trained_model):
  sentences1 = data_type['sent_a']
  sentences2 = data_type['sent_b']
  scores = data_type['score']
  sentences1 = sentences1.tolist()
  sentences2 = sentences2.tolist()
  scores = scores.tolist()
  l1=[]
  l2=[]
  l_cos=[]
  for i in range(len(sentences1)):
    l1.append(trained_model.encode(sentences1[i]))
    l2.append(trained_model.encode(sentences2[i]))

  for i in range(len(l1)):
    l_cos.append(cosine_similarity(l1[i].reshape(1,-1),l2[i].reshape(1,-1))[0][0])
  return l_cos,sentences1,sentences2,scores
  """
  Input the dataset list and return a list of cosine similarity scores. Use the fitted final_trainable_model for obtaining encodings.
  """

train_dl = form_data(df_train)
dev_dl = form_data(df_dev)
test_dl = form_data(df_test)

base_model =  models.Transformer('distilroberta-base',do_lower_case=True,max_seq_length=50)
layer_pooling = models.Pooling(word_embedding_model.get_word_embedding_dimension())
layer_dense = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=50, activation_function=nn.GELU())
model3 = SentenceTransformer(modules=[base_model, layer_pooling, layer_dense])
loss =losses.CosineSimilarityLoss(model=model)



Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
l_cos,sentences1,sentences2,scores=get_model_predicts(df_dev,model3)
print(l_cos)
evaluator1 = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores, write_csv=True, show_progress_bar=True)

# Fit the model3.
model3.fit(train_objectives=[(train_dl, loss)], epochs=2, warmup_steps=100)
# Print spearman correlation on the predicted output of the dev and test sets.
model3.evaluate(evaluator1, output_path='/content')

[0.9983096, 0.99827, 0.9961907, 0.9975654, 0.9970459, 0.99817294, 0.9973494, 0.9967703, 0.996833, 0.9951213, 0.9987614, 0.99635905, 0.99644685, 0.99793863, 0.995707, 0.9934487, 0.99685884, 0.9961814, 0.9952067, 0.99717355, 0.9962623, 0.9973912, 0.99519545, 0.9940367, 0.99632514, 0.99878913, 0.9902032, 0.9962138, 0.9891845, 0.9966487, 0.9959348, 0.99302834, 0.99006075, 0.9905268, 0.99477637, 0.99739903, 0.9957982, 0.9917961, 0.9977916, 0.9938742, 0.99458903, 0.9966995, 0.98372567, 0.9980029, 0.99557316, 0.9910617, 0.9893838, 0.99328905, 0.9937318, 0.99264884, 0.9896021, 0.98735136, 0.995759, 0.9954592, 0.9923837, 0.99303323, 0.99215144, 0.99291915, 0.9941545, 0.9856769, 0.98757935, 0.9875424, 0.99211466, 0.9939214, 0.99340725, 0.989671, 0.9956325, 0.99819225, 0.99280727, 0.9903995, 0.9885752, 0.99282986, 0.9928611, 0.9938849, 0.9928977, 0.99511325, 0.99419713, 0.9944364, 0.9860021, 0.9840904, 0.9959878, 0.99040854, 0.9886656, 0.98698777, 0.997234, 0.9890407, 0.993596, 0.98984694, 0.9853

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

0.6172123362947559

In [ ]:
l_cos,sentences1,sentences2,scores=get_model_predicts(df_test,model3)
print(l_cos)
evaluator2 = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores, write_csv=True, show_progress_bar=True)

# Fit the model3.
model3.fit(train_objectives=[(train_dl, loss)], epochs=1, warmup_steps=100, evaluator=evaluator2, evaluation_steps=500)
# Print spearman correlation on the predicted output of the dev and test sets.
model3.evaluate(evaluator2, output_path='/content')

[0.9983134, 0.9967703, 0.9983835, 0.99802625, 0.99536043, 0.9971836, 0.99776673, 0.9985296, 0.99614066, 0.9973912, 0.9973912, 0.9971382, 0.9958131, 0.9979994, 0.9979179, 0.9973126, 0.99745864, 0.9967229, 0.99647725, 0.9956877, 0.9956837, 0.9931434, 0.9966683, 0.9971034, 0.99668515, 0.99047583, 0.99699646, 0.99631, 0.99725336, 0.993674, 0.99667317, 0.9955385, 0.9937081, 0.9948295, 0.98988414, 0.9909701, 0.9912314, 0.99717754, 0.99478257, 0.99534106, 0.99603724, 0.99505043, 0.9964466, 0.9917506, 0.9947017, 0.98736393, 0.994339, 0.9914409, 0.9897557, 0.9931162, 0.99475366, 0.9952086, 0.9929129, 0.99222, 0.99424386, 0.98773235, 0.9955975, 0.9942767, 0.9956807, 0.98913586, 0.997268, 0.98524404, 0.9937401, 0.9922388, 0.99236894, 0.9932696, 0.99327105, 0.9933225, 0.99347675, 0.9875067, 0.9910371, 0.9914363, 0.98958033, 0.99148506, 0.99212515, 0.98729485, 0.9884848, 0.9889452, 0.98694986, 0.9944316, 0.98026973, 0.98740417, 0.99067986, 0.9885795, 0.9875802, 0.9826343, 0.9929817, 0.9961312, 0.99

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/87 [00:00<?, ?it/s]

Batches:   0%|          | 0/87 [00:00<?, ?it/s]

Batches:   0%|          | 0/87 [00:00<?, ?it/s]

Batches:   0%|          | 0/87 [00:00<?, ?it/s]

0.5221368534703411